# Existential Risk Estimates Database

This notebook processes data from the existential risk (x-risk) estimates database from [this EA Forum post](https://forum.effectivealtruism.org/posts/JQQAQrunyGGhzE23a/database-of-existential-risk-estimates).

Since the entries in the database vary in format and often use natural language, it is necessary to sanitize them for further analysis.

## Pre-Processing

In this step we fiddle with the dataset to get something more readily usable. Among other things, we:
- Create a risk category column, and get rid of the "headers" for things like AI, or Total Risk, etc.
- Remove columns that aren't necessary for the analysis

In [56]:
# Load the data and extract the relevant columns

import pandas as pd
import numpy as np

# Load excel spreadsheet. Load all sheets.
excel = pd.ExcelFile('./data/xrisk-estimates-database-20241204.xlsx')
# List worksheets
excel.sheet_names

['Overall notes',
 'Existential-risk-level estimate',
 'Conditional existential-risk-le',
 'Estimates of somewhat less extr',
 'Other potential estimates or so']

In [57]:
# Use the 'Existential-risk-level estimate' worksheet.
# the first 5 columns are instructions, so skip them.
df = excel.parse('Existential-risk-level estimate', skiprows=5, index_row=0)
df.head()

,Who is the estimator?,When was the estimate made/published?,What is the estimator estimating?,What is their estimate?,Source,Have I properly read the source myself?,Is this estimate included in Beard et al.'s appendix?,Other notes,Unnamed: 8,Unnamed: 9
0,“Total risk” (or similar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",NaN,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,NaN,NaN
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,NaN,NaN
4,"Ben Todd or 80,000 Hours",2017,Extinction risk “in the next century”,Probably at or above 3%,https://80000hours.org/articles/extinction-risk/,Yes,No,NaN,NaN,NaN


In [58]:
# Print number of columns
print(f"DataFrame has {len(df.columns)} columns")

DataFrame has 10 columns


In [59]:
# Print columns
df.columns

Index(['Who is the estimator? ', 'When was the estimate made/published?',
       'What is the estimator estimating?', 'What is their estimate?',
       'Source', 'Have I properly read the source myself?',
       'Is this estimate included in Beard et al.'s appendix?', 'Other notes',
       'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

In [60]:
# Rename columns to something more concise
new_column_names = [
  'estimator',
  'date',
  'estimation_measure',
  'estimation',
  'source',
  'source_read_by_estimator',
  'estimate_included_in_beard_et_al',
  'other_notes',
  'unknown_column_1',
'unknown_column_2'
]
df.columns = new_column_names
df.head()

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,unknown_column_1,unknown_column_2
0,“Total risk” (or similar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",NaN,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,NaN,NaN
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,NaN,NaN
4,"Ben Todd or 80,000 Hours",2017,Extinction risk “in the next century”,Probably at or above 3%,https://80000hours.org/articles/extinction-risk/,Yes,No,NaN,NaN,NaN


In [61]:
# Drop unnecessary columns (last 2, which are unknown)
df = df.drop(columns=['unknown_column_1', 'unknown_column_2'])
df.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes
0,“Total risk” (or similar),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...
18,Buck Shlegris,2023,"""Overall P(doom)"" (from context, this seems to...",0.25,https://youtu.be/YTlrPeikoyw?t=1788,Yes,No,"""My overall P(doom) is like 25% (though it flu..."


In [62]:
# Add empty column for risk category
df['risk_category'] = np.nan
df.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Rohin Shah,2020,"Chance that AI, through “adversarial optimizat...",~5%,https://www.lesswrong.com/posts/TdwpN484eTbPSv...,Yes,No,This is my interpretation of some comments tha...,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,NaN


In [63]:
# The first row indicates what type of risk is being estimated.
# 
# Under the first row are the total risk estimates. 
# 
# Further down below are estimates for other x-risks.
# 
# The categories are: "Total risk (or similar)", "AI", "Biorisk", "Nanotechnology", "Climate Change", "Natural risks (excluding natural pandemics)", "War", "Explicitly about only unrecoverable dystopia and/or unrecoverable collapse (not extinction)", "Miscellaneous".
risk_categories = [
    '“Total risk” (or similar)',
    'AI',
    'Biorisk',
    'Nanotechnology',
    'Climate Change',
    'Natural risks (excluding natural pandemics)',
    'War',
    'Explicitly about only unrecoverable dystopia and/or unrecoverable collapse (not extinction)',
    'Miscellaneous']

risk_categories_aliases = [
    'total',
    'ai',
    'biorisk',
    'nanotechnology',
    'climate_change',
    'natural_risks',
    'war',
    'dystopia',
    'miscellaneous'
]

# Get rows that contain a risk category in the first column
risk_category_rows = df[df['estimator'].isin(risk_categories)]['estimator']
risk_category_rows 

0                             “Total risk” (or similar)
17                                                   AI
39                                              Biorisk
51                                       Nanotechnology
78          Natural risks (excluding natural pandemics)
87                                                  War
92    Explicitly about only unrecoverable dystopia a...
95                                        Miscellaneous
Name: estimator, dtype: object

In [64]:
# Create a copy of the dataframe
df_with_risk_category = df.copy()

# Initialize the current risk category
current_risk_category = None

# Iterate over the dataframe rows
for i, row in df_with_risk_category.iterrows():
  if row['estimator'] in risk_category_rows.values:
    # Update the current risk category
    current_risk_category = row['estimator']
  # Set the risk category for the current row
  df_with_risk_category.at[i, 'risk_category'] = current_risk_category

df_with_risk_category.sample(3)

/var/folders/gp/134vcy4x6c19634zvbkbff900000gn/T/ipykernel_32399/2712489716.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '“Total risk” (or similar)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_with_risk_category.at[i, 'risk_category'] = current_risk_category


,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
64,GCR Conference,2008,Human extinction by 2100 as a result of “nucle...,0.01,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. Beard et al.'s appendix sa...,Nanotechnology
51,Nanotechnology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nanotechnology
9,Gott III,1993,"The prior probability that ""humanity will ceas...",5%.,"Gott III, J. R. (1993). Implications of the Co...",No,Yes,NaN,“Total risk” (or similar)


In [65]:
# Drop the rows that contain the risk categories
df_with_risk_category = df_with_risk_category[~df_with_risk_category['estimator'].isin(risk_categories)]
df_with_risk_category.head(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",“Total risk” (or similar)
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,“Total risk” (or similar)
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,“Total risk” (or similar)


In [66]:
# Rename risk categories to something more concise
df_with_risk_category['risk_category'] = df_with_risk_category['risk_category'].replace(risk_categories, risk_categories_aliases)
df_with_risk_category.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
27,Global Catastrophic Risk Conference,2008,Human extinction by 2100 as a result of “super...,0.05,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. Beard et al.'s appendix sa...,ai
29,Pamlin & Armstrong,2015,"""Infinite impact"" from AI within the next 100 ...",0-10%,"Pamlin, D. & Armstrong, S. (2015). Global Chal...",No,Yes,NaN,ai
89,Will MacAskill,2019/2020,"Likelihood that, if an existential risk in the...",0.9,https://80000hours.org/podcast/episodes/will-m...,Yes,No,"""in terms of my estimates for existential risk...",war


In [67]:
# Use only a few columns relevant for the analysis.
relevant_columns = [
    "estimator",
    "estimation_measure",
    "date",
    "estimation",
    "source_read_by_estimator",
    "risk_category",
    "other_notes"
]
df_relevant = df_with_risk_category[relevant_columns]
df_relevant.head(3)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes
1,Toby Ord,“Total existential risk” by 2120,2020,~17% (~1 in 6),Yes,total,"Ord writes: ""Don’t take these numbers to be co..."
2,GCR Conference,“Overall risk of extinction prior to 2100”,2008,0.19,Yes,total,This is the median. The report about these est...
3,Will MacAskill,Existential risk in the 21st century,2019/2020,0.01,Yes,total,NaN


In [68]:
# Save the dataframe to a CSV file
df_relevant.to_csv('./data/pre-processed_data.csv', index=False)

## Taking out the big guns

This might be total overkill for our purposes, but I find it fun, so, we'll use an LLM to help us process the rest of the columns.




### Estimation measure and estimation

The "estimation measure" varies slightly from estimation to estimation. It will be necessary to harmonize this into something more usable, like current existential risk per century.

For this, I'll attempt using an LLM to help us with this "translation".



In [69]:
# Initialize processor
from utils import RiskEstimateProcessor

processor = RiskEstimateProcessor("./models/llama-2-7b.Q4_K_M.gguf")

llama_load_model_from_file: using device Metal (Apple M1 Pro) - 10915 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_mod

In [70]:
# Load data
df = pd.read_csv('./data/pre-processed_data.csv')
df.sample(10)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes
104,NaN,NaN,NaN,NaN,NaN,miscellaneous,NaN
9,Wells,Annual probability as of 2009 of extinction,2009,0.3-0.4%,No,total,NaN
1,GCR Conference,“Overall risk of extinction prior to 2100”,2008,0.19,Yes,total,This is the median. The report about these est...
73,NaN,NaN,NaN,NaN,NaN,nanotechnology,NaN
38,GCR Conference,Human extinction by 2100 as a result of “the s...,2008,0.0005,Yes,biorisk,This is the median. Beard et al.'s appendix sa...
92,Paul Christiano,Expected fraction of total potential value tha...,2021,0.2,Yes,miscellaneous,Response to Wei Dai's question shown in the ab...
51,GCR Conference,Human extinction by 2100 as a result of “molec...,2008,0.05,Yes,nanotechnology,This is the median. Beard et al.'s appendix sa...
18,Bensinger's survey of 44 people working on lon...,"""How likely do you think it is that the overal...",2021,Mean: ~40%\nMedian: ~30%,Yes,ai,NaN
32,Stuart Armstrong,Existential risk from AI,2020,5-30%,Yes,ai,"""I put the probability that [AI/AGI] is an exi..."
80,"Snyder-Beattie, Ord, & Bonsall",The probability that humanity goes extinct fro...,2019,Almost guaranteed to be below 0.007% (1 in 140...,No,natural_risks,NaN


In [ ]:
# Make an attempt to process risk estimate with a random row 
processed_df = processor.process_dataset(df.sample(1))

In [ ]:

# Now process the whole total risk dataframe
processed_df = processor.process_dataset(df)

Llama.generate: 37 prefix-match hit, remaining 243 prompt tokens to eval
llama_perf_context_print:        load time =    5523.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   243 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    89 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11256.94 ms /   332 tokens
Llama.generate: 37 prefix-match hit, remaining 184 prompt tokens to eval


Processed row 1 of 16


llama_perf_context_print:        load time =    5523.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   184 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    45 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6633.48 ms /   229 tokens
Llama.generate: 40 prefix-match hit, remaining 156 prompt tokens to eval


Processed row 2 of 16


llama_perf_context_print:        load time =    5523.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   156 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    54 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6811.60 ms /   210 tokens
Llama.generate: 37 prefix-match hit, remaining 162 prompt tokens to eval


...

llama_perf_context_print:        load time =    5523.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   201 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    49 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7799.27 ms /   250 tokens
Llama.generate: 38 prefix-match hit, remaining 179 prompt tokens to eval


Processed row 11 of 16


llama_perf_context_print:        load time =    5523.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   179 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    56 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7489.45 ms /   235 tokens
Llama.generate: 37 prefix-match hit, remaining 156 prompt tokens to eval


Processed row 12 of 16


KeyboardInterrupt: 

In [ ]:
processed_df.sample(3)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes,century_probability,conversion_reasoning,conversion_confidence
1,GCR Conference,“Overall risk of extinction prior to 2100”,2008,0.19,Yes,total,This is the median. The report about these est...,0.19,Medium,High
6,Metaculus responders,"""there be zero living humans on planet earth o...",NaN,Median: 1%. Mean: 8%.,No,total,That median and mean is as of 3rd July 2019.,NaN,Error in processing: No JSON object found in r...,low
8,Gott III,"The prior probability that ""humanity will ceas...",1993,5%.,No,total,NaN,NaN,Error in processing: No JSON object found in r...,low


In [ ]:
# Save the processed data to a CSV file.
processed_df.to_csv('./data/risk_estimates_processed.csv', index=False)

In [ ]:
# Validate results
validated_total_risk_df = processor.validate_estimates(processed_df)

# Basic analysis
summary = validated_total_risk_df.groupby('risk_category').agg({
    'century_probability': ['mean', 'median', 'std', 'count'],
    'conversion_confidence': lambda x: (x == 'high').mean()
})

Llama.generate: 3 prefix-match hit, remaining 72 prompt tokens to eval
llama_perf_context_print:        load time =    3682.27 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    72 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    63 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8929.76 ms /   135 tokens
/Users/andresap/repos/curiosity-notebooks/the-extinction-game/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
